In [61]:
import os
import ast
import json
import numpy as np
import seaborn as sns
import pandas as pd
from dance_evaluation import *
import matplotlib.pyplot as plt

In [62]:
mode = "uni"
foot_1S_df = pd.read_csv(f"./aist_pos1s/foot/foot_zero_{mode}_70_145.csv")
foot_2S_df = pd.read_csv(f"./aist_pos2s/foot/foot_zero_{mode}_70_145.csv")

foot_1s_score = pd.read_csv(f"./aist_pos1s/foot/foot_{mode}_70_145_score.csv")
hand_1s_score = pd.read_csv(f"./aist_pos1s/hand/hand_{mode}_70_145_score.csv")

foot_2s_score = pd.read_csv(f"./aist_pos2s/foot/foot_{mode}_70_145_score.csv")
hand_2s_score = pd.read_csv(f"./aist_pos2s/hand/hand_{mode}_70_145_score.csv")

foot_1s_score["hits_idx"] = foot_1s_score["hits_idx"].apply(ast.literal_eval)
hand_1s_score["hits_idx"] = hand_1s_score["hits_idx"].apply(ast.literal_eval)

foot_2s_score["hits_idx"] = foot_2s_score["hits_idx"].apply(ast.literal_eval)
hand_2s_score["hits_idx"] = hand_2s_score["hits_idx"].apply(ast.literal_eval)

total = foot_1S_df.shape[0]

# genre id and total count mapping
json_filename = "genreID_count_mapping.json"
with open(json_filename, "r") as file:
    genre_Tcount = json.load(file)
    
# genre id and name mapping
json_filename = "genre_symbols_mapping.json"
with open(json_filename, "r") as file:
    genre_name = json.load(file)  

In [63]:
def calc_hits(foot_score, hand_score):

    foot_hits = set(foot_score)
    hand_hits = set(hand_score)
    combined_hits = foot_hits.union(hand_hits)
    common_hits = foot_hits.intersection(hand_hits)
    
    print("total:", total)
    print("foot hits:", len(foot_hits), f"({round(len(foot_hits)*100/total, 2)} %)" )
    print("hand hits:", len(hand_hits), f"({round(len(hand_hits)*100/total, 2)} %)" )
    print("combined hits:", len(combined_hits), f"({round(len(combined_hits)*100/total, 2)} %)" )
    print("common hits:", len(common_hits))
    print("foot - common:", len(foot_hits)-len(common_hits))
    print("hand - common:", len(hand_hits)-len(common_hits))
    
    return foot_hits, hand_hits, combined_hits, common_hits


print(10*"*"+f"1-Sensor {mode}"+ 10*"*" )
foot1S_hits, hand1S_hits, combined1S_hits, common1S_hits = calc_hits(foot_1s_score["hits_idx"][3], hand_1s_score["hits_idx"][3])

print(10*"*"+f"2-Sensor {mode}"+ 10*"*" )
foot2S_hits, hand2S_hits, combined2S_hits, common2S_hits = calc_hits(foot_2s_score["hits_idx"][3], hand_2s_score["hits_idx"][3])

**********1-Sensor uni**********
total: 1341
foot hits: 585 (43.62 %)
hand hits: 671 (50.04 %)
combined hits: 909 (67.79 %)
common hits: 347
foot - common: 238
hand - common: 324
**********2-Sensor uni**********
total: 1341
foot hits: 566 (42.21 %)
hand hits: 655 (48.84 %)
combined hits: 861 (64.21 %)
common hits: 360
foot - common: 206
hand - common: 295


In [ ]:
L1 = [hand1S_hits, hand2S_hits]
L1 = [foot1S_hits, foot2S_hits]
L1 = [combined1S_hits, combined2S_hits]

for item in L1:
    hit1S_idx = list(item)
    hit1S_df = foot_1S_df.iloc[hit1S_idx]
    new = hit1S_df.groupby(['dance_genre']).size().to_dict()

    for key, value in new.items():
        print(f"{genre_name.get(key)}:", value, genre_Tcount.get(key), round(value*100/genre_Tcount.get(key)),"%")
    print("\n")

Break: 75 99 76 %
House: 123 136 90 %
Ballet Jazz: 79 136 58 %
Street Jazz: 69 129 53 %
Krump: 81 137 59 %
LA style Hip-hop: 96 141 68 %
Lock: 88 141 62 %
Middle Hip-hop: 128 140 91 %
Pop: 96 141 68 %
Waack: 74 141 52 %


Break: 66 99 67 %
House: 129 136 95 %
Ballet Jazz: 78 136 57 %
Street Jazz: 71 129 55 %
Krump: 79 137 58 %
LA style Hip-hop: 89 141 63 %
Lock: 69 141 49 %
Middle Hip-hop: 115 140 82 %
Pop: 86 141 61 %
Waack: 79 141 56 %




## SAVE CSV

In [52]:
# SAVE TO CSV
L1 = [foot2S_hits, hand2S_hits, combined2S_hits]
file_names = ["./stats/foot2S_hits.csv", "./stats/hand2S_hits.csv", "./stats/combined2S_hits.csv"]

# Save each hit data as a CSV file
for item, file_name in zip(L1, file_names):
    hit1S_idx = list(item)
    hit1S_df = foot_1S_df.iloc[hit1S_idx]  # Ensure foot_1S_df is defined
    new = hit1S_df.groupby(['dance_genre']).size().reset_index(name='count')

    # Add total count and percentage columns
    new['total_count'] = new['dance_genre'].map(genre_Tcount)
    new['percentage'] = round((new['count'] / new['total_count']) * 100, 2)

    # Save to CSV
    new.to_csv(file_name, index=False)